In [12]:
import pandas as pd
import time
import uuid
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

driver = webdriver.Chrome()

sitemap_url = 'https://printmine.in/sitemap_collections_1.xml?from=430672249150&to=477156999486'
driver.get(sitemap_url)

soup = BeautifulSoup(driver.page_source, 'xml')
loc_elements = soup.find_all('loc')
collection_urls = [loc.text for loc in loc_elements if loc.text.startswith('https://printmine.in/collections/')]

products_data = []

def scrape_products(url):
    driver.get(url)
    while True:
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 't4s-product-info'))
            )

            soup_collection = BeautifulSoup(driver.page_source, 'html.parser')
            products_info_divs = soup_collection.find_all('div', class_='t4s-product-info')

            for product_info_div in products_info_divs:
                product_info = {}
                
                title_tag = product_info_div.find('h3', class_='t4s-product-title')
                if title_tag:
                    product_info['title'] = title_tag.text.strip()
                link = product_info_div.find('a').get('href')
                product_info['product_url'] = 'https://printmine.in' + link
                product_url = product_info['product_url']
                driver.get(product_url)
                soup_product = BeautifulSoup(driver.page_source, 'html.parser')
                img_tag = product_info_div.find('img', class_='t4s-lz--fadeIn')
                if img_tag:
                    img_url = img_tag.get('src')
                    if img_url:
                        product_info['image_url'] = img_url
                price_tag = soup_product.find('div', class_='t4s-product-price')
                if price_tag:
                    original_price = price_tag.find('del')
                    discounted_price_tag = price_tag.find('ins')
                    if discounted_price_tag:
                        product_info['discounted_price'] = discounted_price_tag.text.strip()
                    if original_price:
                        product_info['original_price'] = original_price.text.strip()
                
                rating_tag = soup_product.find('div', class_='loox-rating')
                if rating_tag:
                    rating = rating_tag.get('data-rating')
                    product_info['rating'] = rating
                
                products_data.append(product_info)
                driver.back()
                  
            try:

                next_page_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//a[contains(@aria-label, 'Next')]"))
                )
                next_page_url = next_page_button.get_attribute('href')
                if next_page_url:
                    driver.get(next_page_url)
                    time.sleep(2) 
                else:
                    break
            except:

                break
           
        except:

            break


for url in collection_urls:
    print(f"Scraping collection: {url}")
    scrape_products(url)
    
        

print(f"Total products collected: {len(products_data)}")

df = pd.DataFrame(products_data)
df.to_csv('products_info.csv', index=False)


Scraping collection: https://printmine.in/collections/personalized-corporate-gifts-for-all-occasions-shop-printmine-in
Scraping collection: https://printmine.in/collections/illusion-lamps
Scraping collection: https://printmine.in/collections/corporate-awards
Scraping collection: https://printmine.in/collections/personalized-photo-frame-wall-hanging-birthday-anniversary-gift
Scraping collection: https://printmine.in/collections/personalized-pens
Scraping collection: https://printmine.in/collections/3d-miniature-table-top-best-birthday-gift-printmine
Scraping collection: https://printmine.in/collections/personalized-sipper-bottle
Scraping collection: https://printmine.in/collections/personalized-desk-organizers
Scraping collection: https://printmine.in/collections/personalized-keychain-keyrings
Scraping collection: https://printmine.in/collections/buy-religious-gifts
Scraping collection: https://printmine.in/collections/gifts-for-boyfriend
Scraping collection: https://printmine.in/collec